## Freva - Data search and analysis framework for the Community

<img src="Logo.png" with="640px"></img>

<p style="color:black;font-size:18px;">by: Brian Lewis, Christopher Kadow, Etor Lucio Eceiza, Andrej Fast, Martin Bergemann and many others</p>

<h4 style="text-align:center;">Common Problem I: Finding Data</h4>
<img src="Figure_01.jpg" width="640px"></img>

<h4 style="text-align:center;">Common Problem II: Use code of others</h4>
<img src="Figure_02.jpg" width="640px"></img>

<h4 style="text-align:center;">Common Problem III. Remember your own code</h4>
<img src="Figure_02.jpg" width="640px"></img>

* How can we search and access various datasets efficiently?
* How can be streamline user data analysis tools reusable and reproducible?

# Yet another solution - Freva framework

- Freva comes with a three layer architecture

<h4 style="text-align:center; margin-bottom: -1cm;"></h4>
<img src="Concept_Map_1.png" with="540px"></img>

<p style="margin-bottom: -1cm;"></p>
<img src="Concept_Map_2.png" with="540px"></img>

<p style="margin-bottom: -1cm;"></p>
<img src="Concept_Map_3.png" with="540px"></img>

## Example I: Searching and reading CMIP6 data

#### Python call:
```python
import freva
import xarray as xr
files = freva.databrowser(
    project="cmip6",
    experiment="historical",
    model="mpi*",
    ensemble="r1i1*",
    variable="tas",
    time_frequency="mon"
)
dset = xr.open_mfdataset(files, combine="by_coords", parallel=True)
```

#### CLI call:

```bash
⋊  freva databrowser project=cmip6 experiment=historical model='mip*' ensemble='r1i1*' time_frequency=mon variable=tas
```

<h4>Web UI</h4>
<video controls loop height="550">
    <source src="screencast_2.webm"type="video/webm">
    Sorry, your browser doesn't support embedded videos.
</video>

<h4>Inspecting metadata</h4>
<img src="Figure_05.png" with="640px"></img>

## Example II: Data analysis tool / plugins

### Calculate climate change signal and uncertainty in Düren
- Where is the data for the ensemble?
- How to select the municipality area?
- ...

<video controls loop height="650">
    <source src="screencast.webm"type="video/webm">
    Sorry, your browser doesn't support embedded videos.
</video>

<h4>Inspecting the results</h4>
<img src="Figure_06.png" with="640px"></img>

## Setting up data analysis tools

- Freva plugin API is code agnostic
- *Yet*: User defined plugins are defined and within an configured within an python API wrapper:

<p style="margin-bottom: -1cm; margin-top: -1cm;"></p>
```python
from evaluation_system.api import plugin
from evaluation_system.api.parameters import File, ParameterDictionary, SolrField
class PlottingTool(plugin.PluginAbstract):
"""Plugin that plots content of selected data."""
```

<p style="margin-bottom: -0.5cm; margin-top: -0.5cm;"></p>
```python
# Plugin setup
    tool_developer = {"name": "Jane Doe", "email": "Jane.Doe@example.com"}
    __category__ = "support"
    __short_description__ = "Plot data"
    __version__ = (2022, 1, 1)
    __parameters__ = ParameterDictionary(
        File(
            name="in_file",
            file_extension="nc",
            mandatory=True,
            help="Select the input netCDF file.",
        ),
        SolrField(
            name="variable",
            mandatory=False,
            facet="variable",
            help=("Select the variables(s) to be plotted"),
            max_items=1,
        )
    )
```

<p style="margin-bottom: -0.2cm; margin-top: -0.2cm;"></p>
```python
# Workflow definition of the actual tool
 def runTool(self, setup: dict[str, str] = {}) -> None:
        """This method calls the actual plugin."""
        self.call(
          f"python src/plot_map.py {setup['in_file']} -v {setup['variable']}"
        )
```

```bash
⋊  freva plugin -l
Animator: Animate data on lon/lat grids
ClimateChangeProfile: Create climate change signals.
ClimDexCalc: Calculate the ETCCDI climate extreme indices on the
             basis of daily temperature and precipitation data using
             the ClimDex software provided by PCIC.
Climpact: Process climate model data for input of impact model
CWT: Calculate Circulation Weather Type by mean sea level pressure.
EnsemblePlotter: Create and Plot maps of multi-model ensemble means and
                 standard deviations
MoviePlotter: Plots 2D lon/lat movies in GIF format
Papermill: Parametrize a given notebook
PlottingTool: Plot data
```

```bash
⋊  freva plugin plottingtool --doc                                                          
PlottingTool (v2022.1.1): Plot data
Options:
in_file  (default: <undefined>) [mandatory]
         Select the input netCDF file.
variable (default: <undefined>)
         Select the variables(s) to be plotted

```

## Example III: Users adding their own data

```python
import freva
import xarray as xr

user_data = freva.UserData()
user_data.add("eur-11b", "/tmp/my_awesome_data/outfile_?.nc",
              institute="clex", model="UM-RA2T",
              experiment="Bias-correct")
dset = xr.open_mfdataset(freva.databrowser(experiment="bias*", experiment="clex"))
```

## A couple of additional features:
- Users can search their own plugin history and other users results
- Web UI suggests similar results on plugin config
- Full integration of workload managers

## What's next?

* Freva *has* to be come a RestAPI
    * Clients are easy to implement since we only need a couple of REST calls
* We teach freva the Common Workflow Language (cwl) via toil:
```yaml
cwlVersion: v1.0
class: CommandLineTool
hints:
   SoftwareRequirement:
      packages:
      - package: cdo
baseCommand: cdo
```
* We add a system for [ordering/pre-processing data](https://www-regiklim.dkrz.de/plugins/climpact/setup/)
* We add a simple interface for intake
